In [ ]:
%pip install -U azure-ai-ml

In [ ]:
%pip install openai==0.27.8

In [ ]:
import openai
openai.api_type = "azure"
openai.api_base = "https://ailabazopenai.openai.azure.com/"
openai.api_version = "2023-09-15-preview"
openai.api_key = ""

response = openai.ChatCompletion.create(
      engine="ailabgp35turbo",
      messages = [{"role": "system", "content": "You are a receptionst, greeet customer with smile"},
                {"role": "user", "content": "I lost my wallet"}],
      temperature=0.5,
      max_tokens=800,
      top_p=0.95,
      frequency_penalty=0,
      presence_penalty=0,
      stop=None
    )

In [ ]:
print(response['choices'][0]['message']['content'])

In [ ]:
import openai
openai.api_type = "azure"
openai.api_base = "https://ailabazopenaise.openai.azure.com/"
openai.api_version = "2023-09-15-preview"
openai.api_key = ""

response = openai.Completion.create(
      engine="sdk-train-babbage-polysilicon-m01",
      prompt = """The Ministry of New and Renewable Energy (MNRE) has received the President of India’s nod for a new solar energy scheme targeting the electrification of 100,000 unelectrified households in tribal regions with off-grid solar.  
The new scheme for Particularly Vulnerable Tribal Groups (PVTG) Habitations/Villages has been devised under the Pradhan Mantri Janjati Adivasi Nyaya Maha Abhiyan (PM JANMAN). The off-grid solar solution could be through solar home lighting systems (SHLS) or solar mini-grids, based on the results of field surveys. 
Off-grid solar systems will be made available for target households in regions where grid electricity supply is not feasible on a techno-economic basis. The Ministry of Tribal Affairs (MoTA) has identified beneficiary households in 18 states of Andhra Pradesh, Bihar, Chhattisgarh, Gujarat, Jharkhand, Karnataka, Kerala, Madhya Pradesh, Maharashtra, Manipur, Odisha, Rajasthan, Tamil Nadu, Telangana, Tripura, Uttar Pradesh, Uttarakhand and West Bengal, along with the Union Territory of Andaman & Nicobar Islands. 
The central government will cover 100 cost of 0.3 kW solar off-grid system for all 100,000 households, fixed at INR 50,000 ($602)/household. 
Additionally, under the scheme, solar lighting will be provided for 1,500 multi-purpose centers (MOC) in PVTG areas that are off the grid with the central government bearing 100% cost of INR 100,000 ($1,203)/MPC.""",
      temperature=0,
      max_tokens=10,
      top_p=0,
      frequency_penalty=0,
      presence_penalty=0,
      stop=None
    )

In [ ]:
print(response['choices'][0]['text'])

In [2]:
# Import required libraries
from azure.identity import DefaultAzureCredential, InteractiveBrowserCredential

from azure.ai.ml import MLClient, Input, load_component
from azure.ai.ml.constants import AssetTypes
from azure.ai.ml.dsl import pipeline

In [3]:
try:
    credential = DefaultAzureCredential()
    # Check if given credential can get token successfully.
    credential.get_token("https://management.azure.com/.default")
except Exception as ex:
    # Fall back to InteractiveBrowserCredential in case DefaultAzureCredential not work
    credential = InteractiveBrowserCredential()

In [4]:
subscription_id = "f82813e8-b867-48da-9671-d9d20f33bee4"
resource_group = "AILabs-Common-NonProd-HRGPT-RG"
workspace_name = "ailabaml01se"

In [5]:
# Get a handle to registry
ml_client = MLClient(
    credential=credential, registry_name="azureml", registry_location="swedencentral"
)

finetune_pipeline = load_component(
    client=ml_client, name="openai_completions_finetune_pipeline", version="0.1.2"
)

In [6]:
models = ml_client.models.list()
for model in models:
    print(model.name)
    print(model.version)

OpenAI-CLIP-Image-Text-Embeddings-ViT-Large-Patch14-336
None
playground-ayod-rag
None
deci-decidiffusion-v1-0
None
Deci-DeciLM-7B-instruct
None
Deci-DeciLM-7B
None
Deci-DeciCoder-1b
None
mistralai-Mixtral-8x7B-v01
None
mmd-3x-yolof_r50_c5_8x8_1x_coco
None
mmd-3x-vfnet_x101-64x4d-mdconv-c3-c5_fpn_ms-2x_coco
None
mmd-3x-vfnet_r50-mdconv-c3-c5_fpn_ms-2x_coco
None
mmd-3x-sparse-rcnn_r50_fpn_300-proposals_crop-ms-480-800-3x_coco
None
mmd-3x-sparse-rcnn_r101_fpn_300-proposals_crop-ms-480-800-3x_coco
None
mmd-3x-mask-rcnn_swin-t-p4-w7_fpn_1x_coco
None
mmd-3x-deformable-detr_refine_twostage_r50_16xb2-50e_coco
None
microsoft-Orca-2-7b
None
microsoft-Orca-2-13b
None
stabilityai-stable-diffusion-xl-refiner-1-0
None
AutoML-Image-Object-Detection
None
AutoML-Image-Instance-Segmentation
None
AutoML-Image-Classification
None
mmeft
None
ocsort_yolox_x_crowdhuman_mot17-private-half
None
bytetrack_yolox_x_crowdhuman_mot17-private-half
None
mistralai-Mistral-7B-v01
None
mistralai-Mistral-7B-Instruct-v01


In [7]:
print("Pipeline component loaded successfully. Component spec:")
print(finetune_pipeline)

Pipeline component loaded successfully. Component spec:
$schema: http://azureml/sdk-2-0/PipelineComponent.json
name: openai_completions_finetune_pipeline
version: 0.1.2
display_name: OpenAI Completions Finetune Pipeline
description: Finetune your own OAI model. Visit https://learn.microsoft.com/en-us/azure/cognitive-services/openai/
  for more info.
tags:
  contact: gpt3finetuning@microsoft.com
type: pipeline
inputs:
  train_dataset:
    type: uri_folder
    description: Input dataset (file or folder). If a folder dataset is passed, includes
      all nested files.
    optional: false
  validation_dataset:
    type: uri_folder
    description: Input dataset (file or folder). If a folder dataset is passed, includes
      all nested files.
    optional: true
  model:
    type: string
    optional: false
    default: gpt-35-turbo
    description: GPT model engine
    enum:
    - babbage-002
    - davinci-002
    - gpt-35-turbo
    - gpt-4
  task_type:
    type: string
    optional: false


In [8]:
TASK_TYPE = "completion"  # Change this to completion for completion dataset


# Construct pipeline
@pipeline()
def pipeline_with_registered_component(
    train_dataset,
    validation_dataset,
    training_max_epochs=45,
    model="babbage-002",
    registered_model_name="sdk-train-babbage-polysilicon-m03",
    learning_rate_multiplier=1,
    batch_size=15,
    task_type="completion",
):
    openai_completions_finetune_component_results = finetune_pipeline(
        train_dataset=train_dataset,
        validation_dataset=validation_dataset,
        n_epochs=training_max_epochs,
        model=model,
        task_type=task_type,
        registered_model_name=registered_model_name,
        learning_rate_multiplier=learning_rate_multiplier,
        batch_size=batch_size,
    )

    return openai_completions_finetune_component_results


pipeline_job = pipeline_with_registered_component(
    train_dataset=Input(type=AssetTypes.URI_FILE, path="./data/solar_train_dataset.jsonl"),
    validation_dataset=Input(type=AssetTypes.URI_FILE, path="./data/solar_valid_dataset.jsonl"),
    training_max_epochs=45,
    model="babbage-002",  # Select any model from ["babbage-002", "davinci-002", "gpt-35-turbo"]
    task_type=TASK_TYPE,
    registered_model_name="sdk-train-babbage-polysilicon-m03",
    learning_rate_multiplier=1,
    batch_size=15,
)

# set pipeline level compute
pipeline_job.settings.default_compute = "serverless"

In [12]:
subscription_id = "f82813e8-b867-48da-9671-d9d20f33bee4"
resource_group = "AILabs-Common-NonProd-HRGPT-RG"
workspace_name = "ailabaml01se"

In [13]:
workspace_ml_client = MLClient(credential, subscription_id, resource_group, workspace_name)

In [11]:
# Submit pipeline job to workspace
pipeline_job = workspace_ml_client.jobs.create_or_update(
    pipeline_job, experiment_name="sdk-train-pipeline-babbage-polysilicon-m03"
)
pipeline_job

Class AutoDeleteSettingSchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class AutoDeleteConditionSchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class BaseAutoDeleteSettingSchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class IntellectualPropertySchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class ProtectionLevelSchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class BaseIntellectualPropertySchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Uploading solar_train_dataset.jsonl (<

Experiment,Name,Type,Status,Details Page
sdk-train-pipeline-babbage-polysilicon-m03,ashy_wing_dg3lxgjmsb,pipeline,Preparing,Link to Azure Machine Learning studio


In [14]:
import requests
import json
import subprocess

# 1. Add your Azure OpenAI account details
subscription = "f82813e8-b867-48da-9671-d9d20f33bee4"
resource_group = "AILabs-Common-NonProd-HRGPT-RG"
resource_name = "ailabazopenaise" #openai resource name
model_deployment_name = "sdk-train-babbage-polysilicon-m03"

# 2. Add the AzureML registered model name, registered model version, and the AzureML (AML) workspace path for your fine-tuned model.
# Your registered models data can be found in the `Models` tab of your AzureML workspace.
registered_model_name = "sdk-train-babbage-polysilicon-m03"
registered_model_version = "1"
workspace_path = f"/subscriptions/f82813e8-b867-48da-9671-d9d20f33bee4/resourcegroups/AILabs-Common-NonProd-HRGPT-RG/providers/Microsoft.MachineLearningServices/workspaces/ailabaml01se"
#workspace_path = f"/subscriptions/{subscription}/resourcegroups/{resource_group}/providers/Microsoft.MachineLearningServices/workspaces/{workspace_name}"


In [15]:
from azure.identity import DefaultAzureCredential, InteractiveBrowserCredential

try:
    credential = DefaultAzureCredential()
    # Check if given credential can get token successfully.
    access_token = credential.get_token("https://management.azure.com/.default")
    token = access_token.token
except Exception as ex:
    # Fall back to InteractiveBrowserCredential in case DefaultAzureCredential not work
    # This will open a browser page for
    credential = InteractiveBrowserCredential()

In [ ]:
deploy_params = {"api-version": "2023-05-01"}
deploy_headers = {
    "Authorization": f"Bearer {token}",
    "Content-Type": "application/json",
}

# 4. Set model deployment configuration. Here capacity refers to support for `1K Tokens Per Minute (TPM)` for your deployment.
deploy_data = {
    "sku": {"name": "Standard", "capacity": 1},
    "properties": {
        "model": {
            "format": "OpenAI",
            "name": f"{registered_model_name}",
            "version": f"{registered_model_version}",
            "source": f"{workspace_path}",
        }
    },
}

deploy_data = json.dumps(deploy_data)

In [16]:
deploy_params = {"api-version": "2023-05-01"}
deploy_headers = {
    "Authorization": f"Bearer {token}",
    "Content-Type": "application/json",
}

# 4. Set model deployment configuration. Here capacity refers to support for `1K Tokens Per Minute (TPM)` for your deployment.
deploy_data = {
    "sku": {"name": "Standard", "capacity": 50},
    "properties": {
        "model": {
            "format": "OpenAI",
            "name": f"{registered_model_name}",
            "version": f"{registered_model_version}",
            "source": f"{workspace_path}",
        }
    },
}

deploy_data = json.dumps(deploy_data)

In [17]:
# 5. Send PUT request to Azure cognitive services to create model deployment
request_url = f"https://management.azure.com//subscriptions/f82813e8-b867-48da-9671-d9d20f33bee4/resourcegroups/AILabs-Common-NonProd-HRGPT-RG/providers/Microsoft.CognitiveServices/accounts/ailabazopenaise/deployments/sdk-train-babbage-polysilicon-m03"

r = requests.put(
    request_url, 
    params=deploy_params, 
    headers=deploy_headers, 
    data=deploy_data
)

print(r.json())


{'id': '/subscriptions/f82813e8-b867-48da-9671-d9d20f33bee4/resourceGroups/AILabs-Common-NonProd-HRGPT-RG/providers/Microsoft.CognitiveServices/accounts/ailabazopenaise/deployments/sdk-train-babbage-polysilicon-m03', 'type': 'Microsoft.CognitiveServices/accounts/deployments', 'name': 'sdk-train-babbage-polysilicon-m03', 'sku': {'name': 'Standard', 'capacity': 50}, 'properties': {'model': {'format': 'OpenAI', 'name': 'sdk-train-babbage-polysilicon-m03', 'version': '1', 'source': '/subscriptions/f82813e8-b867-48da-9671-d9d20f33bee4/resourcegroups/AILabs-Common-NonProd-HRGPT-RG/providers/Microsoft.MachineLearningServices/workspaces/ailabaml01se'}, 'versionUpgradeOption': 'NoAutoUpgrade', 'capabilities': {'completion': 'true'}, 'provisioningState': 'Creating', 'rateLimits': [{'key': 'request', 'renewalPeriod': 10, 'count': 50}, {'key': 'token', 'renewalPeriod': 60, 'count': 50000}]}, 'systemData': {'createdBy': '30097220@adani.com', 'createdByType': 'User', 'createdAt': '2024-01-10T18:02:2